#San Francisco Crime Prediction 

We are involved in the kaggle.com Crime Prediction contest. We are using the data provided about crimes that occurred in San Francisco from 2011 to 2015. The data are variations on time and location, and the labels on the training set are 39 different crimes. 

To solve this problem we use various models and feature engineering. 

##Initial Attempt

To get on the leaderboard, we did basic feature extraction and normalization, and used a logistric regression model. 

###Initial Feature Engineering Summary

These are the features we get in both training and test data sets.

| Date | DayOfWeek | PdDistrict | Address | Longitude | Latitude |
|---|---|---|---|---|---|---|---|
| 2015-05-10 23:59:00 | Sunday | BAYVIEW | 2000 Block of THOMAS AV | -122.3995877042 | 37.7350510104 |

For the initial attempt, we immediate made these modifications: 

* Semi-normalize years: subtract lowest year, 2011, so that year numbers are from 0 to 4. 
* Separate time information into year, month, day, time of day, day of week
* Normalize latitude and longitude (use (value - mean)/standard deviation)

And chose a logistic regression model for these reasons: 
* Predicting categorical dependent variable
* Relatively large number of samples, not so many features
* Mixture of categorical and continuous data
* Data with obvious correlation (so linear regression would be worse)
* Non-normally distributed data

The results of this attempt are summarized here: 

|  Model | Accuracy | Train Time | Kaggle Score |
|---|---|---|---|---|---|---|---|
| Logistic Regression | 31% | few minutes| 84th |
| Logistic Regression | 30%| few minutes | 105th |



##Data Exploration

SAFYRE? 

Histograms, clustering analysis for decision points and also binarizing/discretizing categorical data

## Feature Engineering: Going Deeper

The initial features shown in the table above are converted into the features described below. The first 6 are based on **Date**, while the last 3 are based on **Address** (more details in sections below).
 
1. **Year**
2. **Month** - Value between 1-12
3. **Day of Month** - Value between 1-31
4. **Hour** - Value between 0-23
5. **Minute of Day** - Value between 0-1439. Calculated by the following formula: ```(hour * 60) + minute``` 
6. **Time of Day** - Categorical feature with the following possible values: Twilight (12 am to 6 am), Morning (6 am to 12 pm), Afternoon (12 pm to 6 pm), Night (6 pm to 12 am)
7. **Day of Week** - Categorical feature with 7 distinct values: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday.
8. **District** - Categorical feature with 10 dictinct values: NORTHERN, PARK, INGLESIDE, BAYVIEW, RICHMOND, CENTRAL, TARAVAL, TENDERLOIN, MISSION, SOUTHERN.
9. **X** - X coordinates as float
10. **Y** - Y coordinates as float
11. **Block** - Categorical variable containing the block number (if not available it's assigned default/empty category). There are 85 distinct values.
12. **Street 1** - Categorical variable containing the first street/avenue/etc. There are 2033 distinct values.
13. **Street 2** - Categorical variable containing the second street/avenue/etc (if not available it's assigned default/empty category). There are 1694 distinct values. 
 
The categorical features, with the exception of the last 3 which we'll discuss later, are converted to dummy variables. That causes the number of features to expand based on the number of distinct values. For example, the **Time of Day** feature is converted to the following 4 features, each with possible values of 0 or 1:

* **IsTwilight**
* **IsMorning**
* **IsAfternoon**
* **IsNight**

As for the features based on **Address** (the last 3), they contained too many distinct values which made it impractical to use with our computers. As such, we first converted each one to a int value (Category1 = 0, Category2, 1, etc.) and then we used the sklearn.preprocessing.OneHotEncoder to convert all of them together into a sparse matrix which made it possible to hold in memory. Afterwards, we reduced the dimensionality to 20 columns using a truncated SVD (aka LSA). We managed to retain more than 44% of the variance.

Finally, we used the sklearn.preprocessing.StandardScaler to do feature scaling on all the numerical features.

## Final set of Features 

These are the features we were left with after feature engineering has been done:

![alt text](features.jpg "Features")


In [1]:
from feature_engineering import *

###########################################
# Prepare data and do feature engineering #
###########################################
fe = FeatureEngineering()
# Used to prepare the data the very first time. This does all of the feature engineering and generates the following
# two CSV files: train_processed.csv and test_processed.csv. After it's been run once on any computer, there's no 
# need to run it again if you just copy those files here.
#fe.prepare_data('train.csv', 'test.csv')

# These methods fetch the pre-processed data stored in train_processed.csv and test_processed.csv.
fe.load_train_data()
# If your PC has less than 8 GB of RAM you might run itno memory issues. If so, you might want to comment this out 
# until you need the data later.
fe.load_test_data()

# Examples of how to access the data (remove later)
print fe.train_data.shape
print fe.train_labels.shape
print fe.mini_train_data.shape
print fe.mini_train_labels.shape
print fe.dev_data.shape
print fe.dev_labels.shape
print fe.test_data.shape
print fe.test_labels.shape
print fe.submission_data.shape # Loaded by load_test_data()

# Examples of how to access categorical features (except Address) and labels.
print fe.districts
print fe.days_of_week
print fe.times_of_day
print fe.labels

Initializing
Loading train data
Shuffle the data and divide it into train, mini_train, dev and test sets
 + Status report:
    - Train data and labels: (580000L, 48L) - (580000L,)
    - Mini train data and labels: (10000L, 48L) - (10000L,)
    - Dev data and labels: (150000L, 48L) - (150000L,)
    - Test data and labels: (148049L, 48L) - (148049L,)

Loading test (submission) data
Converting test data into nparray
 + Status report:
    - Submission data: (884262L, 48L)

(580000L, 48L)
(580000L,)
(10000L, 48L)
(10000L,)
(150000L, 48L)
(150000L,)
(148049L, 48L)
(148049L,)
(884262L, 48L)
['NORTHERN', 'PARK', 'INGLESIDE', 'BAYVIEW', 'RICHMOND', 'CENTRAL', 'TARAVAL', 'TENDERLOIN', 'MISSION', 'SOUTHERN']
['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
['Twilight', 'Morning', 'Afternoon', 'Night']
{'VEHICLE THEFT': 36, 'SUICIDE': 31, 'WEAPON LAWS': 38, 'VANDALISM': 35, 'RECOVERED VEHICLE': 24, 'SECONDARY CODES': 27, 'WARRANTS': 37, 'DRUNKENNESS': 8, 'PROSTITUTION

##Error Analysis

We evaluate a model using: 
* loss function (this is how the kaggle competition scores our submissions)
* f1 score
* classification report (precision, recall)
* confusion matrix 

Given a model, test set and labels, the following function performs an error analysis. 

In [4]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

def print_confusion_heatmap(cm):
    plt.figure(figsize = (15.0, 10.0))
    plt.imshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

def loss_function(probs, samples, num_classes, labels, classes):
    logloss = 0
    for i in range(len(labels)):
        index = classes[labels[i]] # this tells us which probability to take log of 
        p = probs[i, index]
        # to avoid extremes of log function: 
        pn = max(min(p,1-10^(-15)),10^(-15))
        newpart = -(np.log(pn))/samples
        logloss += newpart
    return logloss
        

def error_analysis(model, data, labels, classnames, classes):
    predicted = model.predict(data)
    f1_score = metrics.f1_score(predicted, labels)
    cm = confusion_matrix(labels, predicted)
    samples = data.shape[0] # number of samples - need for loss fcn
    probs = model.predict_proba(data) # predicted probabilities of all classes, all samples
    num_classes = probs.shape[1] # number of classes, need for loss fcn
    loss = loss_function(probs, samples, num_classes, labels, classes)
    
    print "F1 score: %.5f" % f1_score
    print "Cost Function: %.5f" % loss
    print "Classification Report: "
    print(classification_report(labels, predicted, target_names=classnames))
    print_confusion_heatmap(cm)
    

##Variations on Models

In addition to feature engineering, we tried different models and different model parameters. 

###Logistic Regression Experiments

We varied logistric regression models as follows: 
* Penalty terms (we tried L1 and L2)
* Regularization parameter C 
* Convergence tolerance - there is a tradeoff between regularization and convergence; the more regularization, the longer it takes for model fit to converge

Our best model utilizes: 
* L2 penalty (the default)
* C = 0.01 (much higher regularization than the default, which is C= 1.0)
* tol = 0.01 (higher tolerance than the default, which is 0.001)

In [ ]:
lr = LogisticRegression(C=0.01, tol = 0.01)
lr.fit(crimeX, crime_labels)
print "Completed the training"

The results from the `error_analysis` function are shown below. The confusion matrix shows that the logistic regression classifier has a problem. It is predicting mostly on class label frequencies and not taking features so much into account. 

F1 score: 0.32468

Cost Function: 2.67238

Classification Report: 
                             precision    recall  f1-score   support

                    SUICIDE       0.00      0.00      0.00        15
                WEAPON LAWS       0.00      0.00      0.00       843
                    ROBBERY       0.00      0.00      0.00         6
          RECOVERED VEHICLE       0.00      0.00      0.00         1
            SECONDARY CODES       0.00      0.00      0.00       546
      SEX OFFENSES FORCIBLE       0.00      0.00      0.00        42
                   WARRANTS       0.00      0.00      0.00         8
               NON-CRIMINAL       0.00      0.00      0.00       554
               PROSTITUTION       0.00      0.00      0.00        57
              DRUG/NARCOTIC       0.00      0.00      0.00        10
               EMBEZZLEMENT       0.00      0.00      0.00         3
                   TRESPASS       0.00      0.00      0.00       158
    PORNOGRAPHY/OBSCENE MAT       0.00      0.00      0.00       172
                      FRAUD       0.00      0.00      0.00         1
DRIVING UNDER THE INFLUENCE       0.00      0.00      0.00        26
                  LOITERING       0.19      1.00      0.32      1938
                  VANDALISM       0.00      0.00      0.00        23
             MISSING PERSON       0.00      0.00      0.00        16
                   BURGLARY       0.00      0.00      0.00       277
                 BAD CHECKS       0.00      0.00      0.00       931
            STOLEN PROPERTY       0.00      0.00      0.00      1234
                  EXTORTION       0.00      0.00      0.00        70
             SUSPICIOUS OCC       0.00      0.00      0.00       252
                LIQUOR LAWS       0.00      0.00      0.00        24
                      ARSON       0.00      0.00      0.00        67
  SEX OFFENSES NON FORCIBLE       0.00      0.00      0.00        36
                 KIDNAPPING       0.00      0.00      0.00        28
                    BRIBERY       0.00      0.00      0.00         5
              VEHICLE THEFT       0.00      0.00      0.00       301
     FORGERY/COUNTERFEITING       0.00      0.00      0.00        78
                       TREA       0.00      0.00      0.00       527
                    ASSAULT       0.00      0.00      0.00      1188
            FAMILY OFFENSES       0.00      0.00      0.00       467
                DRUNKENNESS       0.00      0.00      0.00        96

                avg / total       0.04      0.19      0.06     10000



<img src="conf_lr.png">



The above confusion matrix shows that the logistic regression classifier has a problem. It is predicting mostly on class label frequencies and not taking features so much into account. 

###Neural Net Experiments

##Ensemble Approaches


###Random Forest Experiments

In [ ]:
clf = RandomForestClassifier(n_estimators=39, class_weight = "auto")
clf.fit(crimeX, crime_labels)
print "Random Forest Training Completed"

Below is the confusion matrix for a subset of 100,000 examples. While the result looks much better than logistic regression, the results are not good on the full test set of over 850,000 examples. The loss function computed on the test set was lower than 1.0, but the loss function on the full test set was 6. 

<img src="conf_randomF.png">

###Simple Logistic Regression and Random Forest Ensemble

This area shows some promise. The initial random forest results had loss function values around 6, and logistic regression around 2.7 We tried several simple combinations of logistic regression and random forest models, and these are an improvement on both. Results are summarized here: 

|Predicted Probability | Loss Function |
|-----|-----|
|Average of LR and RF | 2.53 on full test set |
|Maximum of LR and RF | 2.55 on full test set |

Neither of these did as well as the neural net models. Some sample code of how the ensemble is generated is below. 

In [ ]:
lr_probs = lr.predict_proba(mini_testX)
clf_probs = clf.predict_proba(mini_testX)
av_probs = (lr_probs + clf_probs)/2
print av_probs.shape

##Results and Conclusions

ALL - let's merge what we have. 